In [ ]:
# !pip install transformers
# !pip install torch
!pip install transformers torch faiss-cpu pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 30.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-ma

In [ ]:
import torch
import pandas as pd
from transformers import BertTokenizer, AutoModel
from torch.utils.data import DataLoader, Dataset
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
from transformers import BertTokenizer, AutoModel
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-lite-base-p1")
model = AutoModel.from_pretrained("indobenchmark/indobert-lite-base-p1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


pytorch_model.bin:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

In [ ]:
dataset = pd.read_csv('final_dataset.csv')

In [ ]:
dataset = dataset.iloc[:, [1, 2]]
# # dataset.columns = ['question', 'answer']
# # dataset = dataset.dropna(subset=['question', 'answer'])

In [ ]:
dataset.head()

,question,answer
0,"baik, kalau kamu bagaimana?",aku baik-baik saja juga. makasih udah nanyain ...
1,aku baik-baik saja juga. makasih udah nanyain ...,"ga masalah, bagaimana hari ini?"
2,"ga masalah, bagaimana hari ini?",hari ini berjalan dengan baik. kalo kamu gimana?
3,hari ini berjalan dengan baik. kalo kamu gimana?,hariku juga baik hari ini. aku sedang di kampu...
4,hariku juga baik hari ini. aku sedang di kampu...,kamu kuliah di mana?


In [ ]:
questions = dataset["question"].tolist()
answers = dataset["answer"].tolist()

In [ ]:
num_layers_to_use = 3
model.config.num_hidden_layers = num_layers_to_use

In [ ]:
import numpy as np

def compute_embeddings(texts, model, tokenizer, num_layers):
    embeddings = []
    for text in texts:
        input_ids = tokenizer.encode(text, return_tensors='pt')
        with torch.no_grad():
            outputs = model(input_ids, output_hidden_states=True)
            text_embeddings = torch.stack(outputs.hidden_states[-num_layers:]).mean(dim=0).mean(dim=1)
            embeddings.append(text_embeddings.squeeze().numpy())
    return np.array(embeddings)


In [ ]:
precomputed_embeddings = compute_embeddings(questions, model, tokenizer, num_layers_to_use)

In [ ]:
import faiss
# Step 4: Build a Faiss index for fast similarity search

# Determine the dimensionality of embeddings
d = precomputed_embeddings.shape[1]  # Dimension of embeddings

# Initialize Faiss index with L2 distance metric
index = faiss.IndexFlatL2(d)

# Add precomputed embeddings to the Faiss index
index.add(precomputed_embeddings)


In [ ]:
def generate_response(input_text):
    # Tokenize and encode input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Get BERT model embeddings for input text
    with torch.no_grad():
        outputs = model(input_ids, output_hidden_states=True)
        input_embeddings = torch.stack(outputs.hidden_states[-num_layers_to_use:]).mean(dim=0).mean(dim=1)

    # Convert tensor to numpy array for faiss
    input_embeddings = input_embeddings.squeeze().numpy().reshape(1, -1)

    # Search for the nearest neighbor
    D, I = index.search(input_embeddings, 1)  # Search for the closest match
    best_response_idx = I[0][0]
    best_response = answers[best_response_idx]

    return best_response


In [ ]:
print("\nBot: Selamat datang! Anda bisa keluar dari percakapan dengan mengetik 'exit'.")

while True:
    user_input = input("Anda: ")
    if user_input.lower() == 'exit':
        print("Bot: Selamat tinggal!")
        break
    response = generate_response(user_input)
    print("Bot:", response)



Bot: Selamat datang! Anda bisa keluar dari percakapan dengan mengetik 'exit'.
Anda: halo
Bot: hai, ada apa?
Anda: exit
Bot: Selamat tinggal!


In [ ]:
torch.save(model.state_dict(), "/content/optimized_bert_model.pth")

# Save the faiss index
faiss.write_index(index, "/content/faiss_index.bin")

print("\nModel and index saved successfully.")


Model and index saved successfully.


# CONVERT KE TF

In [ ]:
config = model.config
config.num_hidden_layers = num_layers_to_use
config.output_hidden_states = True

In [ ]:
import torch.nn as nn
from transformers.convert_graph_to_onnx import convert

In [ ]:
class CustomBertModel(nn.Module):
    def __init__(self, config, model):
        super(CustomBertModel, self).__init__()
        self.config = config
        self.model = model

    def forward(self, input_ids, attention_mask=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.hidden_states[-num_layers_to_use:]
        return hidden_states

custom_model = CustomBertModel(config, model)

In [ ]:
torch.save(custom_model.state_dict(), "/content/modified_bert_model.pth")

In [ ]:
import tensorflow as tf
from transformers import TFAutoModel

In [ ]:
tf_model = TFAutoModel.from_pretrained("indobenchmark/indobert-lite-base-p1", config=config)

tf_model.h5:   0%|          | 0.00/63.1M [00:00<?, ?B/s]

Some layers from the model checkpoint at indobenchmark/indobert-lite-base-p1 were not used when initializing TFAlbertModel: ['predictions', 'sop_classifier']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at indobenchmark/indobert-lite-base-p1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


In [ ]:
tf_model.save("/content/tf_optimized_bert_model")